In [1]:
from experta import *
from Fact import *

In [2]:
# Global Variables
case_of_injury              = 'FALSE'      # TRUE, FALSE, SUSPICION
doubt_counter               = 0
risk_of_injury              = 0.0

In [3]:
class FirstCheck( KnowledgeEngine ):

    @Rule( Change_in_breast('yes')|Tissue_in_breast('yes') | Breast_withdrawal('yes')  )
    def Tissue_in_breast(self):
        global case_of_injury
        case_of_injury = 'TRUE'

    @Rule(Pain_in_armpit_and_breast('yes'))
    def Paininarmpitand_breast(self):
        global doubt_counter
        doubt_counter += 2

    @Rule(Redness_of_breast_skin('yes'))
    def Redness_of_breast_skin(self):
        global doubt_counter
        doubt_counter += 2

    @Rule(Itching_or_rash('yes'))
    def Itching_or_rash(self):
        global doubt_counter
        doubt_counter += 2

    @Rule(Dump_the_nipple('yes'))
    def Dump_the_nipple(self):
        global doubt_counter
        doubt_counter += 2

    @Rule(Peeling_or_falling_skin('yes'))
    def Peeling_or_falling_skin(self):
        global doubt_counter
        doubt_counter += 2
        
        
class SecondCheck( KnowledgeEngine ):

    @Rule(Age( P(lambda x: x < 30 )))
    def age_case_1(self):
        global risk_of_injury
        risk_of_injury += 0.05

    @Rule(Age( P(lambda x: x < 50) & P(lambda x: x >= 30) ))
    def age_case_2(self):
        global risk_of_injury
        risk_of_injury += 2.0

    @Rule(Age(P(lambda x: x >= 50)))
    def age_case_3(self):
        global risk_of_injury
        risk_of_injury += 7.7

    @Rule(weight(P(lambda x: x >= 70)))
    def weight(self):
        global risk_of_injury
        risk_of_injury += 15.0
        
    @Rule(Genetics('yes'))
    def genetics(self):
        global risk_of_injury
        risk_of_injury += 7.0

    @Rule(Previous_injury('yes'))
    def previousInjury(self):
        global risk_of_injury
        risk_of_injury += 40.0

    @Rule(Hormonal_compensatory_therapy(temp = 'x' << W()))
    def hormonalCompensatoryTherapy(self, x):
        global risk_of_injury
        if x == 1:
            risk_of_injury += 34.0
        elif x == 2:
            risk_of_injury += 43.0
        elif x == 3:
            risk_of_injury += 73.0

    @Rule(Contraceptive_pills(temp='x' << W()))
    def contraceptivePills(self, x):
        global risk_of_injury
        if x == 1:
            risk_of_injury += 9.0
        elif x == 2:
            risk_of_injury += 38.0

    @Rule(Breast_feeding(temp='x' << W(), stat='y' << W()))
    def breastFeeding(self, x, y):
        global risk_of_injury
        if y == 'yes':
            risk_of_injury -= float(x) * 4.0 * 2.0

            



In [4]:
FirstCheck = FirstCheck()
FirstCheck.reset()

In [5]:
print('HELLO...\n We\'ll ask some questions, and you must answer with yes or no please :)')
print('---------------------------------------------------------------------------------------------------')
FirstCheck.declare( Tissue_in_breast(input('Is there a thick tissue in the breast, or a lump in the breast or armpit ? ')) )
FirstCheck.declare( Breast_withdrawal(input('Is there a withdrawal of the nipple into the breast, or any other parts of the breast ? ')) )
FirstCheck.declare( Change_in_breast(input('Is there a change in the shape or size of the breast ? ')) )
FirstCheck.declare( Pain_in_armpit_and_breast(input('Is there pain in the armpit and breast that does not change with the menstrual cycle ? ')) )
FirstCheck.declare( Redness_of_breast_skin(input('Is there a redness in the skin of the breast ( and turning orange color ) ? ')) )
FirstCheck.declare( Itching_or_rash(input('Is there itching or rash on the nipples or on one of them ? ')) )
FirstCheck.declare( Dump_the_nipple(input('Is there a discharge from the nipple ? ')) )
FirstCheck.declare( Peeling_or_falling_skin(input('Is there a peeling or falling of the skin of the breast ? ')) )
FirstCheck.run()

if case_of_injury == 'TRUE' and doubt_counter >= 6:
    case_of_injury = 'TRUE'
else:
    case_of_injury = 'FALSE' if doubt_counter == 0 else 'SUSPICION'

# ================================
#      This's the second check
# ================================
if case_of_injury != 'TRUE':
    SecondCheck = SecondCheck()
    SecondCheck.reset()
    
    
    print('\nNow We\'ll ask you a general questions during your life, and you must answer depending on the question pattern')
    print('---------------------------------------------------------------------------------------------------')
    # This questions aimed at knowing the factors that will increase the incidence of the disease in the future
    SecondCheck.declare(Age(int(input('How old are you ? '))))
    SecondCheck.declare(weight(int(input('What is your weight ? '))))
    SecondCheck.declare(Genetics(input('Is there an infection with this disease in the immediate family ?(yes,no) ')))
    
    if input( 'Have you make a DNA test and the gene ( brca1 ) has been mutated ?(yes,no) ') == 'yes':
        risk_of_injury += 60.0
    else: 
        risk_of_injury += 0.0
    if input( 'Have you make a "MARKERS" test ? (yes,no)') == 'yes':
        if int(input('what was it persent ?\n' )) > 30.5 :
               risk_of_injury +=20.7
        else :
               risk_of_injury +=0.0
    else: 
        risk_of_injury += 0.0
    #risk_of_injury += 0.0 if input( 'Have you make a DNA test and the gene ( brca1 ) has been mutated ? ') == 'YES' else 60.0
    SecondCheck.declare(Previous_injury(input('Is there a previous infection of the disease and was so serious that it spread to the lymphoma ? (yes,no)')))
    if input('Have you been exposed to hormone therapy ?(yes,no) ') == 'yes':
        print('Choose your approval status : ')
        temp = int(input('1 - Treatment by Estrogen.\n' + '2 - Treatment by Estrogen & Progestin.\n' + '3 - Hormone therapy has been taken for more than five years.\n'))
        SecondCheck.declare(Hormonal_compensatory_therapy( temp = temp))
    if input('Have you taken the contraceptive pills before ? (yes,no)') == 'yes':
        print('Choose your approval status : ')
        temp = int(input('1 - These pills were taken for a period of no more than one year.\n' + '2 - These pills were taken for more than 10 years.'))
        SecondCheck.declare(Contraceptive_pills( temp = temp))

    
    if input('Are you married ? (yes,no)') == 'yes':
        if input('Do you have children ?(yes,no) ') == 'yes':
            temp = input('How many ? ')
            if input('Have you fully breastfed your children ?(yes,no) ') == 'yes':
                SecondCheck.declare(Breast_feeding(temp = temp, stat = 'yes'))
            else:
                SecondCheck.declare(Breast_feeding(temp = temp, stat = 'no'))
        else:
            print('well done!')
    else:
        print('----------------------------------------')
        print('WELL WE ARE FINISHED OUR QUESTIONS')
    SecondCheck.run()
    if case_of_injury == 'SUSPICION':
            print('\nThere\'s Suspicion of infection in the rate of ' + str(doubt_counter * 10) + '%')
    else:
            print('\nThere\'s no tumor')

    if risk_of_injury > 90.0:
            print('Unfortunately!! There is a very high risk of future injury')
    elif risk_of_injury <= 0.0:
            print('There is no risk of future injury')
    else:
            print('The possibility of future injury is ' + str(risk_of_injury) + '%')
            
else:
    print('\n IT,S TUMOR.')

HELLO...
 We'll ask some questions, and you must answer with yes or no please :)
---------------------------------------------------------------------------------------------------



Now We'll ask you a general questions during your life, and you must answer depending on the question pattern
---------------------------------------------------------------------------------------------------


ValueError: invalid literal for int() with base 10: ''